# rosalie dev

Notebook purpose:

- Develop `rosalie` package and templates for working with it

In [69]:
import logging
import os

import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu

import rosalie as ro
import fabutils as fu


pd.set_option('display.max_rows', 500)

%config InlineBackend.figure_format ='retina'
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
df = fu.dgp.dgp_cuped().make_data(n=10_000_000, seed=2312).set_index('id', drop=False)
df.head()

,id,y,x,d
id,,,,
0,0,23.871381,16.416167,1
1,1,19.789949,17.460931,1
2,2,26.884960,22.942863,0
3,3,23.529556,17.170929,1
4,4,20.878394,17.922234,0


In [72]:
def welch_t_test(df, metric, d="d"):
    """Return p-value of Welch's t-test."""
    control_sample = df.query("assignments == 'control'")[metric]
    variant_sample = df.query("assignments == 'treatment'")[metric]
    return ttest_ind(control_sample, variant_sample, equal_var=False).pvalue

def mww(df, metric, d="d"):
    """Return p-value of Mann-Whitney-Wilcoxson U-test."""
    control_sample = df.query("assignments == 'control'")[metric]
    variant_sample = df.query("assignments == 'treatment'")[metric]
    return mannwhitneyu(control_sample, variant_sample).pvalue


## testing

In [101]:
%%timeit

eval = ro.Simulator(
    df=df,
    metrics=['y'],
    evaluators=[welch_t_test],
    sample_min=100,
    sample_max=1_000,
    num_steps=4,
    mdes=[0.02],
    num_runs=20,
    verbose=False,
)
result = eval.run()

100%|██████████| 4/4 [00:04<00:00,  1.11s/it]

5.75 s ± 253 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
%%timeit

eval = ro.Simulator(
    df=df,
    metrics=['y'],
    evaluators=[welch_t_test],
    sample_min=100,
    sample_max=1_000,
    num_steps=4,
    mdes=[0.02],
    num_runs=20,
    verbose=False,
)
result = eval.run()

100%|██████████| 4/4 [00:00<00:00,  5.98it/s]

1.84 s ± 94.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## dev

In [91]:
%lprun -f eval.run eval.run()

eval = ro.Simulator(
    df=df,
    metrics=['y'],
    evaluators=[welch_t_test],
    sample_min=100,
    sample_max=1_000,
    num_steps=4,
    mdes=[0.02],
    num_runs=20,
    verbose=True,
)
result = eval.run()

100%|██████████| 4/4 [00:01<00:00,  3.81it/s]
INFO - Initializing Simulator with specified evaluators: ['welch_t_test']
INFO - Initializing Simulator with specified evaluators: ['welch_t_test']
INFO - Initializing Simulator with specified evaluators: ['welch_t_test']
INFO - Initializing Simulator with specified evaluators: ['welch_t_test']
100%|██████████| 4/4 [00:00<00:00,  5.97it/s]


Timer unit: 1e-09 s

Total time: 1.05953 s
File: /Users/fgu/dev/personal/rosalie/rosalie/simulator.py
Function: run at line 117

Line #      Hits         Time  Per Hit   % Time  Line Contents
   117                                               def run(self):
   118                                                   """Run simulation."""
   119         1       1000.0   1000.0      0.0          results = []
   120         2       2000.0   1000.0      0.0          for preprocessor in self.preprocessors:
   121         1       2000.0   2000.0      0.0              df_preproc = preprocessor(self.df)
   122         5    8345000.0    2e+06      0.8              for sample_size in tqdm(self._calc_sample_sizes()):
   123         4      10000.0   2500.0      0.0                  sample_func = self._get_sample_func()
   124         4    5077000.0    1e+06      0.5                  df_sample = sample_func(df_preproc, sample_size)
   125        84      51000.0    607.1      0.0                  for